# Importacoes

In [ ]:
import tensorflow as tf


In [ ]:
print(tf.__version__)


In [ ]:
import requests
import pandas as pd
import numpy as np


# Conectando com servidor

In [ ]:
diaInicio = '20230801'
diaFinal = '20230131'
idGrupo = '12'
idRegra = '106'
idTipo = 'null'
url = 'http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoSaidas/listaParaTreino?diaNumInicio={}&diaNumFinal={}&idGrupoAcao={}&idRegraProjecao={}&idTipoExemplo={}'.format(diaInicio,diaFinal,idGrupo,idRegra,idTipo)


In [ ]:
print(url)


In [ ]:
r = requests.get(url)
j = r.json()
df = pd.DataFrame.from_dict(j)


In [ ]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')


In [ ]:
dfTratadoX = df.apply(trataLinhaX, axis='columns')
X = np.stack(dfTratadoX.values)
X.shape


In [ ]:
Y = np.stack(df['campoY'].values)
Y = Y.reshape(-1,1)
Y.shape


# Dados Validacao

In [ ]:
diaInicio = '20230131'
diaFinal = '20230131'
idGrupo = '12'
idRegra = '106'
idTipo = 'null'
urlValida = 'http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoSaidas/listaParaTreino?diaNumInicio={}&diaNumFinal={}&idGrupoAcao={}&idRegraProjecao={}&idTipoExemplo={}'.format(diaInicio,diaFinal,idGrupo,idRegra,idTipo)


In [ ]:
print(urlValida)


In [ ]:
r = requests.get(urlValida)
j = r.json()
dfValida = pd.DataFrame.from_dict(j)


In [ ]:
dfValida.head()


In [ ]:
dfTratadoValidaX = dfValida.apply(trataLinhaX, axis='columns')
XValida = np.stack(dfTratadoValidaX.values)
XValida.shape


In [ ]:
YValida = np.stack(dfValida['campoY'].values)
YValida = YValida.reshape(-1,1)
YValida.shape


# Minha Rede

## Estrutra

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu',input_shape=(1800,1)))
model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.summary()


## Compilacao

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
                optimizer=RMSprop(learning_rate=1e-4),
                metrics=['accuracy'])


## Fit - Treino

In [ ]:
X2 =  X[...,None]
XValida2 =  XValida[...,None]
history = model.fit(x=X2,y=Y, validation_data=(XValida2, YValida) ,
                    epochs=100,
                    verbose=2)


## Salva Rede

In [ ]:
path='pesos/treino_rede886.h5'
model.save(path)


# Enviando Rede

In [ ]:
!pip install paramiko


In [ ]:
import os
import paramiko


In [ ]:
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect("191.252.219.136", username="root", password="Digicom2004$")
sftp = ssh.open_sftp()
sftp.put("pesos/treino_rede886.h5", "/root/executa-rede/redes/treino_rede886.h5")
sftp.close()
ssh.close()


# Registra Treinamento

In [ ]:
#import requests
url = 'http://vps-40d69db1.vps.ovh.ca:23004/api/TreinoRedes/executouTreinamento'
myobj = {'id': 886}
x = requests.post(url, json = myobj)
print(x.text)
